In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import resample
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.optimizers import Adam
import pickle

In [2]:
# Initializing the path of the working directory
base_dir = r"C:\Users\MSI\Documents\MiniProject"

# Mapping the image id with their path so that it can be referred easily
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x 
                     for x in glob(os.path.join(base_dir, 'sample', '*.jpg'))}

# This dictionary is useful for displaying more human-friendly labels later on
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}


In [3]:
# Reffering to the medadata having the information about all the images we have.
skin_df = pd.read_csv(os.path.join(base_dir, 'HAM10000_metadata.csv'))

# Creating New Columns path , cell_type and cell_type_idx for better readability

# Keeping the path of the respected image
skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)

# Keeping the human understable Name with the respected name
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get)

# Keeping the index of the cell_type of the dict
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [4]:
skin_df['cell_type'].value_counts()

Melanocytic nevi                  6705
Melanoma                          1113
Benign keratosis-like lesions     1099
Basal cell carcinoma               514
Actinic keratoses                  327
Vascular lesions                   142
Dermatofibroma                     115
Name: cell_type, dtype: int64

In [5]:
skin_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\MSI\Documents\MiniProject\sample\ISIC...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\MSI\Documents\MiniProject\sample\ISIC...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\MSI\Documents\MiniProject\sample\ISIC...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\MSI\Documents\MiniProject\sample\ISIC...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,C:\Users\MSI\Documents\MiniProject\sample\ISIC...,Benign keratosis-like lesions,2


In [6]:
skin_df.isnull().sum()

lesion_id         0
image_id          0
dx                0
dx_type           0
age              57
sex               0
localization      0
dataset           0
path              0
cell_type         0
cell_type_idx     0
dtype: int64

In [7]:
skin_df['age'].fillna((skin_df['age'].mean()) , inplace = True)

In [8]:
skin_df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age              0
sex              0
localization     0
dataset          0
path             0
cell_type        0
cell_type_idx    0
dtype: int64

In [9]:
cell_type_idx = 'cell_type_idx'
record_0 = skin_df[skin_df[cell_type_idx] == 0]
record_1 = skin_df[skin_df[cell_type_idx] == 1]
record_2 = skin_df[skin_df[cell_type_idx] == 2]
record_3 = skin_df[skin_df[cell_type_idx] == 3]
record_4 = skin_df[skin_df[cell_type_idx] == 4]
record_5 = skin_df[skin_df[cell_type_idx] == 5]
record_6 = skin_df[skin_df[cell_type_idx] == 6]

In [10]:
n = 500
record_bal_0 = resample(record_0 , replace = True , n_samples = n , random_state = 42)
record_bal_1 = resample(record_1 , replace = True , n_samples = n , random_state = 42)
record_bal_2 = resample(record_2 , replace = True , n_samples = n , random_state = 42)
record_bal_3 = resample(record_3 , replace = True , n_samples = n , random_state = 42)
record_bal_4 = resample(record_4 , replace = True , n_samples = n , random_state = 42)
record_bal_5 = resample(record_5 , replace = True , n_samples = n , random_state = 42)
record_bal_6 = resample(record_6 , replace = True , n_samples = n , random_state = 42)

In [11]:
balance_record = pd.concat([record_bal_0 , record_bal_1 , record_bal_2 , record_bal_3 , record_bal_4 , record_bal_5 , record_bal_6])

In [12]:
balance_record['image'] = balance_record['path'].map(lambda x: np.asarray(Image.open(x).resize((64,64))))

In [13]:
X = np.asarray(balance_record['image'].tolist())

In [14]:
Y = balance_record['cell_type_idx']

In [15]:
Y_cat = to_categorical(Y)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.20,random_state=42)

In [17]:
x_train_mean = np.mean(x_train)
x_test_mean = np.mean(x_test)

x_train_std = np.std(x_train)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_test_mean) / x_test_std

In [40]:
input_shape = (64 , 64 , 3)

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size = (2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),activation='relu'))
model.add(MaxPool2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 64, 64, 64)        1792      
                                                                 
 conv2d_25 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 30, 30, 128)       73856     
                                                                 
 conv2d_27 (Conv2D)          (None, 28, 28, 256)       295168    
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 14, 14, 256)      0         
 g2D)                                                 

In [41]:
model.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics=['accuracy'])

In [42]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

datagen.fit(x_train)

In [43]:
epochs = 50
batch_size = 16
history = model.fit(datagen.flow(x_train , y_train  , batch_size = batch_size) , epochs = epochs , validation_data=(x_test , y_test) , verbose = 1)

Epoch 1/50
175/175 [==============================] - 139s 785ms/step - loss: 1.7525 - accuracy: 0.2971 - val_loss: 1.5718 - val_accuracy: 0.3886
Epoch 2/50
175/175 [==============================] - 136s 778ms/step - loss: 1.5056 - accuracy: 0.4121 - val_loss: 1.4366 - val_accuracy: 0.4171
Epoch 3/50
175/175 [==============================] - 136s 780ms/step - loss: 1.4134 - accuracy: 0.4446 - val_loss: 1.5367 - val_accuracy: 0.4143
Epoch 4/50
 32/175 [====>.........................] - ETA: 1:43 - loss: 1.4369 - accuracy: 0.4512

KeyboardInterrupt: 

In [39]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("cnn3.h5")

22/22 [==============================] - 1s 23ms/step - loss: 0.8944 - accuracy: 0.7129
Test: accuracy = 0.712857  ;  loss = 0.894366


In [ ]:
#1. Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1, len(model_history.history['accuracy'])/10))
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
   #  axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1, len(model_history.history['loss'])/10))

    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
plot_model_history(history)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
y_predict_classes = np.argmax(prediction , axis=1)

In [ ]:
y_predict_classes

In [ ]:
y_true = np.argmax(y_test , axis=1)

In [ ]:
y_true

In [ ]:
# Function to plot confusion matrix    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_predict_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(7))